## Workshop 2: The Pooling Problem

Exercises Adapted from: https://jckantor.github.io/cbe30338-book/notebooks/milk-pooling.html?highlight=pooling

### 1. Installing and Importing Packages 

We first need to pull in all the packages we will be using. Pyomo is a Python-based, open-source optimization modelling language with a diverse set of optimization capabilities. For more information, see the Pyomo [documentation](https://pyomo.readthedocs.io/en/stable/).

In [ ]:
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np
import pandas as pd
from ipywidgets import FloatSlider, interact
import platform
import os 

# Solver setup for Windows or Linux
def setup_solver():
    os_name = platform.system()
    if os_name == "Windows":
        return "solver/ipopt.exe", "solver/cbc.exe", "solver/ampl.mswin64/bonmin.exe"
    elif os_name == "Linux":
        os.system("chmod +x solver/ipopt")
        os.system("chmod +x solver/cbc")
        os.system("chmod +x solver/bonmin")
        return "solver/ipopt", "solver/cbc", "solver/bonmin"

ipopt_executable, cbc_executable, bonmin_executable = setup_solver()

### 2. Blending Milk and Distribution

![image.png](images\milk-pooling_simple.png)


The milk distributor blends supplies from the two local farms to meet customer requirements. Let $L$ designate the set of local suppliers, and let $C$ designate the set of customers. The decision variable $x_{l,c}$ is the amount of milk from local supplier $l\in L$ mixed into the blend for customer $c\in C$.

The distributor’s objectives is to maximize profit:

\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
\end{align*}
 
where the notation $(l,c) \in L \times C$ indicates a summation over the cross-product of two sets. A useful interpretation is that $(l,c) \in L \times C$ describes all ways of delivering milk from $l$ to $c$. Each term ($\text{price}_{c} - \text{price}_{l}$) is then the profit earned by delivering one unit of milk from $l \in L$ to $c \in C$.

The amount of milk delivered to each customer can not exceed customer demand.

\begin{align*}
\sum_{l\in L} x_{l, c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}
 		
The milk blend delivered to each customer  must meet the minimum product quality requirement for milk fat. Assuming linear blending, the model becomes:

\begin{align*}
\sum_{(l,c)\ \in\ L \times C} \text{conc}_{l} x_{l,c} & \geq \text{conc}_{c} \sum_{l\in L} x_{l, c} & \forall c \in C
\end{align*}	

In [ ]:
customers = pd.DataFrame({
    "Customer A": {"fat": 0.0445, "price": 52.0, "demand": 6000.0},
    "Customer B": {"fat": 0.030, "price": 48.0, "demand": 2500.0}
}).T

suppliers = pd.DataFrame({
    "Farm A": {"fat": 0.045, "cost": 45.0, "location": "local"},
    "Farm B": {"fat": 0.030, "cost": 42.0, "location": "local"},
    "Farm C": {"fat": 0.033, "cost": 37.0, "location": "remote"},
    "Farm D": {"fat": 0.050, "cost": 45.0, "location": "remote"}},
    ).T

local_suppliers = suppliers[suppliers["location"]=="local"]
remote_suppliers = suppliers[suppliers["location"]=="remote"]

print("\nCustomers")
print(customers)

print("\nLocal Suppliers")
print(local_suppliers)

print("\nRemote Suppliers")
print(remote_suppliers)

#### a. Option 1 (LP): Current Setup - Two Local Milk Farms, Two Customers

In [ ]:
def initialize_model(local_suppliers, customers):
    model = ConcreteModel()
    model.L = Set(initialize=local_suppliers.index.tolist())   # Local suppliers
    model.C = Set(initialize=customers.index.tolist())         # Customers
    model.x = Var(model.L * model.C, domain=NonNegativeReals)  # Flow rate variables
    return model

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;">
    <span style="color:blue;"><b>EXERCISE:</b> Write a function named <code>add_objective_function()</code> that adds the objective function to the model to maximize profit, as shown in the above formulation.</span>
</div>


\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
\end{align*}

In [ ]:
def add_objective_function(model, suppliers, customers):
    # Define the profit objective as a simple expression

    profit_expr = pass # EXERCISE: Complete the expression for the profit

    # Assign this expression directly as the objective
    model.profit = Objective(expr=profit_expr, sense=maximize)
    return model

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;">
    <span style="color:blue;"><b>EXERCISE:</b> Implement a function called <code>add_demand_constraints()</code> that ensures the total flow of milk delivered to each customer does not exceed their demand.</span>
</div>

\begin{align*}
\sum_{l\in L} x_{l, c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}

In [ ]:
def add_demand_constraints(model, customers):
    # HINT: Define the demand constraints directly in a loop
    model.demand_constraints = ConstraintList()
    
    # EXERCISE: add a demand constraint for each customer
    return model

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;">
    <span style="color:blue;"><b>EXERCISE:</b> Create a function named <code>add_quality_constraints()</code> that ensures the blended milk delivered to each customer meets their minimum fat content requirement.</span>
</div>

\begin{align*}
\sum_{(l,c)\ \in\ L \times C} \text{conc}_{l} x_{l,c} & \geq \text{conc}_{c} \sum_{l\in L} x_{l, c} & \forall c \in C
\end{align*}	

In [ ]:
def add_quality_constraints(model, suppliers, customers):
    #EXERCISE: add quality constraint for each customer
    return model

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;">
    <span style="color:blue;"><b>EXERCISE:</b> Use the pre-built <code>solve_model()</code> function to run the optimisation problem.</span>
</div>

In [ ]:
def solve_model(model):
    solver = SolverFactory('cbc', executable=cbc_executable) 
    results = solver.solve(model)
    
    print(f"\nTotal Profit = £{model.profit():.0f}\n")  # Display the total profit
    print("Optimal Flow Rates:")                        # Display flowrates
    for l in model.L:
        for c in model.C:
            flow_rate = model.x[l, c].value
            print(f"Flow from {l} to {c}: {flow_rate}")

In [ ]:
# Run the complete model
model = initialize_model(local_suppliers, customers)
model = add_objective_function(model, suppliers, customers)
model = add_demand_constraints(model, customers)
model = add_quality_constraints(model, suppliers, customers)
solve_model(model)

### 2. Pooling and Blending Milk and Distribution

#### a. Option 2 (MINLP): Potential New Setup - Four Milk Farms, Two Customers

![image.png](images\milk-pooling.dio.png)

The distributor wants to see if additional profit can be made by purchasing raw milk two remote farms. This would require purchasing and operating a separate delivery truck. Blending is also not possible in the remote locations.

The question is whether the one existing truck could transport a pool of raw milk from remote farms C and D that could be blended with raw milk from local farms A and B to meet customer requirements. The profit potential may be reduced due to pooling, but it may still be a better option than adding additional operating expense.

The pooling problem is famous, and there are a number of formulations used to model it. We will use a version of the "p-parameterisation" where composition of the pool is a decision variable. Here that variable will be called $f$.

Other decision variables are also needed. Decision variables $y_c$ refer to the amount of the pool used in the blend delivered to customer $c\in C$. Variables $z_r$ ar the amount of raw milk purchased from remote farm $r$ and included in the pool.

#### Updated Problem Formulation 

In the objective function, there are new terms for the cost of raw milk included in the pool, and customer revenue earned from use of the pool. 
$$
\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
+ \sum_{c\in C} \text{price}_c y_{c} - \sum_{r\in R} \text{cost}_r z_{r}
\end{align*}
$$

The product delivered to each customer can not exceed customer demand.

$$
\begin{align*}
\sum_{l\in L} x_{l, c} + y_{c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}
$$

Incoming and outgoing flows to the pool must balance.

$$
\begin{align*}
\sum_{r\in R}z_{r} & = \sum_{c\in C} y_{c} \\
\end{align*}
$$

The milk fat composition of the pool, $f$, is given by averaging contributions from the remote farms.

$$
\begin{align*}
\sum_{r\in R}\text{conc}_{r} z_{r}  & = \underbrace{f \sum_{c\in C} y_{c}}_{\text{bilinear}}
\end{align*}
$$

Finally, the minimum milk fat required by each customer $c\in C$ satisfies a blending constraint.

$$
\begin{align*}
\underbrace{f y_{c}}_{\text{bilinear}}  + \sum_{(l,c)\ \in\ L \times C} x_{l,c} \text{conc}_{l} 
& \geq \text{conc}_{c} (\sum_{l\in L} x_{l, c} + y_{c})
& \forall c \in C
\end{align*}
$$

The last two constraints include bilinear terms from the project of decision variable $f$ with decision variables $y_c$ for all $c\in C$. 


In [ ]:
def initialize_model(local_suppliers, remote_suppliers, customers):
    m = ConcreteModel()
    
    # Define sets for local and remote suppliers and customers
    m.L = Set(initialize=local_suppliers.index)   # Local suppliers
    m.R = Set(initialize=remote_suppliers.index)  # Remote suppliers
    m.C = Set(initialize=customers.index)         # Customers
    
    # Define decision variables
    m.x = Var(m.L * m.C, domain=NonNegativeReals)             # Flow rates from local suppliers to customers
    m.y = Var(m.C, domain=NonNegativeReals)                   # Flow rates from the pool to customers
    m.z = Var(m.R, domain=NonNegativeReals)                   # Flow rates from remote suppliers to the pool
    m.f = Var(bounds=(0.03, 0.051), domain=NonNegativeReals)  # Pool composition of fat content
    
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Define the objective function to maximize profit using <code>add_objective_function()</code>. Hint: perhaps think about separately calculating the local profit, pool profit and remote cost and then use these terms to calculate the total profit</span> </div>

$$
\begin{align*}
\text{Profit} & = \sum_{(l,c)\ \in\ L \times C} (\text{price}_c - \text{cost}_l) x_{l,c}
+ \sum_{c\in C} \text{price}_c y_{c} - \sum_{r\in R} \text{cost}_r z_{r}
\end{align*}
$$

In [ ]:
def add_objective_function(m, suppliers, customers):
    # EXERCISE: Calculate profits from local suppliers to customers

    # EXERCISE: Calculate profit from pooled supply to customers

    # EXERCISE: Calculate cost of remote suppliers contributing to the pool

    # Define the objective function
    m.profit = Objective(expr=local_profit + pool_profit - remote_cost, sense=maximize)
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Define the customer demand constraints using <code>add_demand_constraints()</code>.</span> </div>

$$
\begin{align*}
\sum_{l\in L} x_{l, c} + y_{c} & \leq \text{demand}_{c} & \forall c\in C
\end{align*}
$$

In [ ]:
def add_demand_constraints(m, customers):
    # EXERCISE: add demand constraints for all customers
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Define the pool balance constraint using <code>add_pool_balance_constraint()</code>.</span> </div>

$$
\begin{align*}
\sum_{r\in R}z_{r} & = \sum_{c\in C} y_{c} \\
\end{align*}
$$

In [ ]:
def add_pool_balance_constraint(m):
    # EXERCISE: add pool balance constraint
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Define the pool quality constraint using <code>add_pool_quality_constraint()</code>.</span> </div>

$$
\begin{align*}
\sum_{r\in R}\text{conc}_{r} z_{r}  & = \underbrace{f \sum_{c\in C} y_{c}}_{\text{bilinear}}
\end{align*}
$$

In [ ]:
def add_pool_quality_constraint(m, suppliers):
    # EXERCISE add the pool quality constraint
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Define the customer quality constraints using <code>add_customer_quality_constraints()</code>.</span> </div>

$$
\begin{align*}
\underbrace{f y_{c}}_{\text{bilinear}}  + \sum_{(l,c)\ \in\ L \times C} x_{l,c} \text{conc}_{l} 
& \geq \text{conc}_{c} (\sum_{l\in L} x_{l, c} + y_{c})
& \forall c \in C
\end{align*}
$$


In [ ]:
def add_customer_quality_constraints(m, suppliers, customers):
    # EXERCISE: add the customer quality constraint for all customers
    return m

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Solve the model and print the results using <code>solve_model()</code>.</span> </div>

In [ ]:
def solve_model(m):
    solver = SolverFactory('bonmin', executable=bonmin_executable)
    results = solver.solve(m)
    print("Optimal Profit: £{:.0f}".format(value(m.profit)))

<div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; font-size: 1.2em;"> <span style="color:blue;"><b>EXERCISE:</b> Run the cell below to solve the optimisation problem and print the result</code>.</span> </div>

In [ ]:
# Initialize the model and add constraints step by step
m = initialize_model(local_suppliers, remote_suppliers, customers)
m = add_objective_function(m, suppliers, customers)
m = add_demand_constraints(m, customers)
m = add_pool_balance_constraint(m)
m = add_pool_quality_constraint(m, suppliers)
m = add_customer_quality_constraints(m, suppliers, customers)

# Solve the model and display results
solve_model(m)